# EDA

## Getting Data

In [3]:
# getting access to data_prep.py functions inside flood_prediction package
from flood_prediction.ml_logic.data_prep import *

In [4]:
df = get_data_and_targets()
df.head()

Getting Weather Data
<Response [200]>
Getting Flood Data


,date,T(degC),rain(mm),surf_press(hPa),wind_s(km/h),wind_dir(deg),soil_moist_0_to_7cm(m3),soil_moist_7_to_28cm(m3),radiation(W/m2),river_discharge(m3/s),target
0,1984-01-01 00:00:00,19.2,0.0,982.1,4.5,331,0.251,0.278,0.0,48.77,0
1,1984-01-01 01:00:00,19.1,0.0,981.9,4.5,331,0.251,0.278,0.0,48.77,0
2,1984-01-01 02:00:00,18.6,0.0,982.0,4.1,345,0.251,0.278,0.0,48.77,0
3,1984-01-01 03:00:00,18.2,0.0,981.9,3.6,6,0.251,0.278,0.0,48.77,0
4,1984-01-01 04:00:00,17.7,0.0,981.9,3.7,11,0.251,0.278,0.0,48.77,0


## Basic EDA

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347832 entries, 0 to 347831
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date                      347832 non-null  datetime64[ns]
 1   T(degC)                   347832 non-null  float64       
 2   rain(mm)                  347832 non-null  float64       
 3   surf_press(hPa)           347832 non-null  float64       
 4   wind_s(km/h)              347832 non-null  float64       
 5   wind_dir(deg)             347832 non-null  int64         
 6   soil_moist_0_to_7cm(m3)   347832 non-null  float64       
 7   soil_moist_7_to_28cm(m3)  347832 non-null  float64       
 8   radiation(W/m2)           347832 non-null  float64       
 9   river_discharge(m3/s)     347809 non-null  float64       
 10  target                    347832 non-null  int64         
dtypes: datetime64[ns](1), float64(8), int64(2)
memory usage: 29.2 MB


In [6]:
round(df.describe().transpose(),2)

,count,mean,std,min,25%,50%,75%,max
T(degC),347832.0,16.13,6.87,-6.80,11.10,15.60,20.70,37.80
rain(mm),347832.0,0.05,0.33,0.00,0.00,0.00,0.00,9.60
surf_press(hPa),347832.0,983.10,3.09,965.10,981.00,982.80,984.90,998.40
wind_s(km/h),347832.0,5.86,3.77,0.00,3.00,4.90,8.00,31.10
wind_dir(deg),347832.0,205.24,91.21,1.00,129.00,225.00,275.00,360.00
soil_moist_0_to_7cm(m3),347832.0,0.34,0.09,0.25,0.25,0.30,0.42,0.52
soil_moist_7_to_28cm(m3),347832.0,0.34,0.08,0.28,0.28,0.30,0.42,0.52
radiation(W/m2),347832.0,236.03,327.02,0.00,0.00,10.00,446.00,1149.00
river_discharge(m3/s),347809.0,36.64,40.52,4.66,17.45,26.16,40.98,1010.58
target,347832.0,0.00,0.05,0.00,0.00,0.00,0.00,1.00


In [7]:
df.isnull().sum()

date                         0
T(degC)                      0
rain(mm)                     0
surf_press(hPa)              0
wind_s(km/h)                 0
wind_dir(deg)                0
soil_moist_0_to_7cm(m3)      0
soil_moist_7_to_28cm(m3)     0
radiation(W/m2)              0
river_discharge(m3/s)       23
target                       0
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.isnull().sum()

date                        0
T(degC)                     0
rain(mm)                    0
surf_press(hPa)             0
wind_s(km/h)                0
wind_dir(deg)               0
soil_moist_0_to_7cm(m3)     0
soil_moist_7_to_28cm(m3)    0
radiation(W/m2)             0
river_discharge(m3/s)       0
target                      0
dtype: int64

In [10]:
# counting amount of target = 1
df['target'].value_counts()

0    346873
1       936
Name: target, dtype: int64

In [18]:
# printing when target is 1
df[df['target']==1][['date','river_discharge(m3/s)','target']][::24]

,date,river_discharge(m3/s),target
4416,1984-07-03,179.72,1
4440,1984-07-04,1010.58,1
4464,1984-07-05,673.53,1
21528,1986-06-16,552.35,1
21552,1986-06-17,612.76,1
21576,1986-06-18,317.42,1
31656,1987-08-12,549.81,1
31680,1987-08-13,436.52,1
31704,1987-08-14,565.24,1
48768,1989-07-25,70.61,1


## Is this the correct threshold? Are we missing any disaster date?

In [22]:
# threshold for river discharge= 240 m3/s
# printing every time river discharge is above threshold from 2015 to 2023 and target is 0 (grabbing every 24 rows to make it only 1 per day)
df[(df['river_discharge(m3/s)']>180)][['date','river_discharge(m3/s)','target']][::24].tail(25)

,date,river_discharge(m3/s),target
197808,2006-07-26,381.82,0
197832,2006-07-27,209.38,0
199704,2006-10-13,283.45,0
205560,2007-06-14,222.36,0
213816,2008-05-23,234.50,0
213912,2008-05-27,286.74,0
213936,2008-05-28,184.78,0
215856,2008-08-16,330.13,0
224592,2009-08-15,294.80,0
225120,2009-09-06,330.89,0
